# Proyecto Pronóstico y predicción

La cadena de gimnasios Model Fitness está desarrollando una estrategia de interacción con el cliente basada en datos analíticos.
Uno de los problemas más comunes que enfrentan los gimnasios y otros servicios es la pérdida de clientes. ¿Cómo descubres si un cliente ya no está contigo? Puedes calcular la pérdida en función de las personas que se deshacen de sus cuentas o no renuevan sus contratos. Sin embargo, a veces no es obvio que un cliente se haya ido: puede que se vaya de puntillas.
Los indicadores de pérdida varían de un campo a otro. Si un usuario compra en una tienda en línea con poca frecuencia pero con regularidad, no se puede decir que ha huido. Pero si durante dos semanas no ha abierto un canal que se actualiza a diario, es motivo de preocupación: es posible que tu seguidor se haya aburrido y te haya abandonado.
En el caso de un gimnasio, tiene sentido decir que un cliente se ha ido si no viene durante un mes. Por supuesto, es posible que estén en Cancún y retomen sus visitas cuando regresen, pero ese no es un caso típico. Por lo general, si un cliente se une, viene varias veces y luego desaparece, es poco probable que regrese.
Con el fin de combatir la cancelación, Model Fitness ha digitalizado varios de sus perfiles de clientes. Tu tarea consiste en analizarlos y elaborar una estrategia de retención de clientes.
Tienes que:
- Aprender a predecir la probabilidad de pérdida (para el próximo mes) para cada cliente
- Elaborar retratos de usuarios típicos: selecciona los grupos más destacados y describe sus características principales
- Analizar los factores que más impactan la pérdida
- Sacar conclusiones básicas y elaborar recomendaciones para mejorar la atención al cliente:
    - Identificar a los grupos objetivo
    - Sugerir medidas para reducir la rotación
    - Describir cualquier otro patrón que observes con respecto a la interacción con los clientes


In [1]:
# Librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

## Tabla de contenidos <a id='back'></a>

1. [Descarga de datos y preparación](#descarga_datos)
2. [Análisis exploratorio de datos (EDA)](#eda)
3. [Modelo para predecir la cancelación de usuarios](#modelo_cancelacion)
4. [Clústeres de usuarios](#clusters)
5. [Conclusiones y recomendaciones](#conclusiones_recomendaciones)

## Descarga de datos y preparación <a id="descarga_datos"></a>

Vamos a cargar los datos del dataset `gym_churn_us.csv`

In [4]:
# Carga los archivos de datos
data = pd.read_csv('../data/gym_churn_us.csv')

# Verificamos que los datos se hayan cargado correctamente
data.head()

,gender,Near_Location,Partner,Promo_friends,Phone,Contract_period,Group_visits,Age,Avg_additional_charges_total,Month_to_end_contract,Lifetime,Avg_class_frequency_total,Avg_class_frequency_current_month,Churn
0,1,1,1,1,0,6,1,29,14.227470,5.0,3,0.020398,0.000000,0
1,0,1,0,0,1,12,1,31,113.202938,12.0,7,1.922936,1.910244,0
2,0,1,1,0,1,1,0,28,129.448479,1.0,2,1.859098,1.736502,0
3,0,1,1,1,1,12,1,33,62.669863,12.0,2,3.205633,3.357215,0
4,1,1,1,1,1,1,0,26,198.362265,1.0,3,1.113884,1.120078,0


Vamos a estudiar los datos que contiene nuestro dataset y verificar que los tipos de datos sean correctos para nuestro análisis

In [5]:
# Imprimimos la información general/resumen sobre nuestro dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   gender                             4000 non-null   int64  
 1   Near_Location                      4000 non-null   int64  
 2   Partner                            4000 non-null   int64  
 3   Promo_friends                      4000 non-null   int64  
 4   Phone                              4000 non-null   int64  
 5   Contract_period                    4000 non-null   int64  
 6   Group_visits                       4000 non-null   int64  
 7   Age                                4000 non-null   int64  
 8   Avg_additional_charges_total       4000 non-null   float64
 9   Month_to_end_contract              4000 non-null   float64
 10  Lifetime                           4000 non-null   int64  
 11  Avg_class_frequency_total          4000 non-null   float

Nuestro dataset `data` que son los datos sobre la cancelación de un mes en concreto e información del mes que lo precedía de Model Fitness, contiene los siguientes datos:

- `Churn`: la cancelación para el mes en cuestión
- Campos de dataset actuales:
    - `gender`: género del cliente
    - `Near_Location`: si el usuario vive o trabaja en el vecindario donde se encuentra el gimnasio
    - `Partner`: si el usuario es un trabajador de una compañía asociada (el gimnasio tiene empresas asociadas cuyos empleados obtienen descuentos; en esos casos el gimnasio almacena información sobre los empleadores de los clientes)
    - `Promo_friends`: si el usuario originalmente se inscribió mediante una oferta "trae a un amigo" (se utilizó el código promocional de un amigo cuando pagaron el primer abono)
    - `Phone`: si el usuario aportó el número de teléfono
    - `Age`: edad del cliente
    - `Lifetime`: el tiempo (en meses) desde que el usuario llegó por primera vez al gimnasio
- Datos del registro de visitas y compras y datos sobre el estado actual de la membresía
    - `Contract_period`: periodo del contrato (1 mes, 3 meses, 6 meses o 1 año)
    - `Month_to_end_contract`: los meses que faltan hasta que expire el contrato
    - `Group_visits`: si el usuario participa en sesiones grupales
    - `Avg_class_frequency_total`: frecuencia media de visitas por semana a lo largo de la vida del cliente
    - `Avg_class_frequency_current_month`: frecuencia media de visitas por semana durante el mes en curso
    - `Avg_additional_charges_total`: cantidad total de dinero gastado en otros servicios del gimnasio: cafetería, productos deportivos, cosméticos, masajes, etc.

In [6]:
# Verificamos que no haya datos duplicados en el dataset
print('Datos duplicados en el dataset:', data.duplicated().sum())

Datos duplicados en el dataset: 0


In [7]:
# Se imprime una descripción de los datos numericos
data[['Age', 'Lifetime', 'Contract_period', 'Month_to_end_contract', 'Avg_class_frequency_current_month', 'Avg_class_frequency_total', 'Avg_additional_charges_total']].describe()

,Age,Lifetime,Contract_period,Month_to_end_contract,Avg_class_frequency_current_month,Avg_class_frequency_total,Avg_additional_charges_total
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,29.184250,3.724750,4.681250,4.322750,1.767052,1.879020,146.943728
std,3.258367,3.749267,4.549706,4.191297,1.052906,0.972245,96.355602
min,18.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.148205
25%,27.000000,1.000000,1.000000,1.000000,0.963003,1.180875,68.868830
50%,29.000000,3.000000,1.000000,1.000000,1.719574,1.832768,136.220159
75%,31.000000,5.000000,6.000000,6.000000,2.510336,2.536078,210.949625
max,41.000000,31.000000,12.000000,12.000000,6.146783,6.023668,552.590740


#### Conclusiones del dataset

Al ver la muestra y resumen del dataset, no encontramos duplicados ni ausentes. Sin embargo es necesario ajustar los nombres de las columnas al estilo snake_case para facilitar el uso y llamado en procesos posteriores, tambien es necesario ajustar las columnas con opciones 1 y 0 a `boolean` para optimizar su uso. Por último, gracias a la descripción de los datos númericos vemos que la mayoria de las columnas no presentan datos outliers, las unicas a tener cierta precaución serian `contract_period`, `month_to_end_contract` y `avg_additional_charges_total` ya que sus medias tienen diferencias un poco significativas con sus medianas.

### Corregir datos

In [ ]:
# Cambiar los nombres de las columnas
data.rename(columns={"Near_Location": "near_location", "Partner": "partner", "Promo_friends": "promo_friends"}, inplace=True)
# Cambiar el tipo de dato de chain de object a bool
data_rest['chain'] = data_rest['chain'].astype('bool')
data_rest.info()